In [ ]:
from pyvis.network import Network
import networkx as nx
import pandas as pd
import glob 

In [ ]:
def calculate_step_sizes (df) :
    feedback_quantities = df['feedback quantities'].str.split(',', expand=True).astype(int)
    step_sizes = []
    for col in feedback_quantities :
        tmp = sorted(feedback_quantities[col].unique())
        if len(tmp) < 2 :
            step_sizes.append(0)
        else :
            step_sizes.append(tmp[1] - tmp[0])
            
    return step_sizes


def split_list_in_dataframe(df, colname) :
    new_df = df.drop(columns=[colname]).copy()
    new_cols = df[colname].str.split(',', expand=True).astype(int)
    new_col_names = [f"{colname}_{x}" for x in range(len(new_cols.columns))]
    new_df[new_col_names] = new_cols.copy()
    return new_df

In [ ]:
def compute_next(step_sizes) :
    def inner_compute_next(row) :
        criticals = list(map(int, [x for x in row["critical feedback"].split(',') if x != ""]))
        originals = list(map(int, [x for x in row["feedback quantities"].split(',') if x != ""]))
        res = []
        for critical_buffer_idx in criticals :
            critical_buffer_idx = critical_buffer_idx - 1
            next_configuration = originals.copy()
            if step_sizes[critical_buffer_idx] > 0 :
                next_configuration[critical_buffer_idx] = next_configuration[critical_buffer_idx] + step_sizes[critical_buffer_idx]
                res.append(",".join(map(str, next_configuration)))
        return res
    return inner_compute_next

def turn_config_to_index(df) :
    df_by_fq = df.reset_index(names="index").set_index("feedback quantities")

    def inner_turn_config_to_index(row) :
        next_configs = row["next configurations"]
        next_configs_indexes = []
        for config in next_configs :
            try :
                next_configs_indexes.append(df_by_fq.loc[config]["index"])
            except KeyError :
                pass
        # list(map(lambda x : df_by_fq.loc[x]["index"], ))
        return next_configs_indexes
    return inner_turn_config_to_index

In [ ]:
def load_one_file (filename) :
    
    # load one particular file
    import numpy as np 

    df = pd.read_csv(filename, dtype = {"throughput" : np.float64,
                                                         "storage distribution size": np.int64,
                                                         "feedback quantities" : str,
                                                         "critical feedback" : str,
                                                        })
    df = df[["throughput","storage distribution size","feedback quantities","critical feedback"]]
    df["critical feedback"] = df["critical feedback"].fillna("")

    # Given the step sizes, and critical feedbackm we can compute the next configurations, 
    # handy to generate the graph properly.
    step_sizes = calculate_step_sizes(df)
    df["next configurations"] = df.apply(compute_next(step_sizes),axis=1)
    df["next configurations"] = df.apply(turn_config_to_index(df),axis=1)

    tmp_df = df.reset_index(names="index")
    df["past configurations"] = tmp_df.apply(lambda x :  [idx for idx,y in enumerate(tmp_df["next configurations"]) if x["index"] in y] , axis = 1)


    # Compute X and Y position for the visualization.
    df['Y'] = 50*(df['storage distribution size'].diff(1).shift(0) > 0).cumsum()
    df['X'] = 400*df.groupby('storage distribution size').cumcount()

    df["interesting"] = df.apply(lambda x : x["throughput"] > df.loc[[y for y in x["past configurations"]]]["throughput"].max() , axis=1)
    return df

In [ ]:
import pyvis
#97c2fc80
UNUSED_COLOR='#97c2fc80'
USED_COLOR='#97c2fc'
INTERESTING_COLOR='#ff5733'

def generate_config_graph(df):
    
    step_sizes = calculate_step_sizes(df)
    print(step_sizes)
    
    feedback_quantities = df['feedback quantities'].str.split(',', expand=True).astype(int)
    
    col_of_interest = [x for x in feedback_quantities.columns if step_sizes[x] > 0 ]
    feedback_quantities = feedback_quantities[col_of_interest]
    step_sizes = [x for x in step_sizes if x > 0]
    
    names_feedback_quantities = feedback_quantities[col_of_interest].apply(lambda x : ",".join(x.astype(str)),axis=1)
    
    #  
    # Create a directed graph
    net = Network(notebook=True)
    # Add nodes and edges to the graph
    for i, row1 in feedback_quantities.iterrows():
        
        # decide of the color
        interesting = df.iloc[i]["interesting"] 
        color = USED_COLOR
        if interesting :
            color = INTERESTING_COLOR
        
        net.add_node(i,physics=False, 
                     x=int(df.iloc[i]["X"]), 
                     y=int(df.iloc[i]["Y"]), 
                     label=str(df.iloc[i]["storage distribution size"]) + "(" + str(df.iloc[i]["throughput"])+ ")", 
                     title=f"{names_feedback_quantities.iloc[i]}\n{df.iloc[i]}", 
                     shape='box', 
                     borderWidth=3 if i == 0 else 0,
                     color=color,
                    )  # Set size to 20 for the first node, 10 for others
     
    for i, row1 in feedback_quantities.iterrows():
        for j in df.iloc[i]["next configurations"]:
            net.add_edge(i, int(j), width = 2)
                

    return net

In [ ]:
import re 

kdse_logs = {}

#Load all the files
filenames = [x for x in glob.glob('../kdse2023_log/*__athroughputbufferingDSE__prealtime_1__pmode_*.txt')]

for f in filenames :
    matching = re.match('.*/(.*)__athroughputbufferingDSE__prealtime_1__pmode_(.*).txt', f)
    if matching :
        name = matching.group(1)
        method = matching.group(2)
        if not name in kdse_logs:
            kdse_logs[name] = {}
        kdse_logs[name][method] = f        
kdse_logs

In [ ]:
df = load_one_file(kdse_logs["fig8"]["KDSE"])
net = generate_config_graph(df)
net.show("interactive_network.html")

In [ ]:
df = load_one_file(kdse_logs["fig8"]["K2DSE"])
net = generate_config_graph(df)
net.show("interactive_network.html")

In [ ]:
df = load_one_file(kdse_logs["fig8"]["K2DSEA"])
net = generate_config_graph(df)
net.show("interactive_network.html")